In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub

import nltk
from nltk.probability import FreqDist 
import os
print(os.listdir("../input"))

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Any results you write to the current directory are saved as output.


2025-08-18 22:33:01.582639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755556381.596474       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755556381.600803       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 22:33:01.619394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


['test.csv.zip', 'sample_submission.csv', 'train.csv', 'test.csv', 'sample_submission.csv.zip', 'description.md', 'train.csv.zip', 'train.zip', 'spooky-author-identification']


In [3]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")


AttributeError: module 'tensorflow_hub' has no attribute 'Module'

In [4]:
train = pd.read_csv('../input/train.csv')
train.head()


,id,text,author
0,id06121,So I did not abandon the search until I had be...,EAP
1,id01074,He had promised to spend some hours with me on...,MWS
2,id12145,In the end he give him a funny kind o' thingum...,HPL
3,id18286,What other construction could I possibly put u...,EAP
4,id11118,"I had heard the noise, however faint, and my s...",EAP


In [5]:
test = pd.read_csv('../input/test.csv')
test.head()


,id,text
0,id27251,"There seemed to be a void, and nothing more, a..."
1,id09612,This event caused many of those who were about...
2,id11943,"I hastily gave my consent to this arrangement,..."
3,id19526,It were useless to record the debate that foll...
4,id12931,"But how are we to judge of airs, and pronounce..."


In [6]:
sns.countplot(train['author']);


ValueError: could not convert string to float: 'EAP'

In [7]:
x = list(train['text'])
y = list(train['author'])


In [8]:
le = LabelEncoder()
le.fit(y)
le.classes_


array(['EAP', 'HPL', 'MWS'], dtype='<U3')

In [9]:
def encode(le, label):
    enc = le.transform(label)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)


In [10]:
encD =  encode(le, ['EAP'])
encD


array([[1.]])

In [11]:
decD = decode(le, encD)
decD


array(['EAP'], dtype='<U3')

In [12]:
x_enc = x
y_enc = encode(le , y)
y_enc


array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [13]:
x_train = np.asarray(x_enc[:15000])
y_train = np.asarray(y_enc[:15000])

x_test = np.asarray(x_enc[15000:])
y_test = np.asarray(y_enc[15000:])


In [14]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Lambda, Input
import keras.backend as K


In [15]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))


In [16]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512,))(input_text)
dense1 = Dense(256, activation='relu')(embedding)
dropout = Dropout(0.4)
dense2 = Dense(256, activation='relu')(dense1)
pred = Dense(3, activation='softmax')(dense2)
model = Model([input_text], outputs=pred )

model.summary()


2025-08-18 22:33:08.534385: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,891 (773.01 KB)

 Trainable params: 197,891 (773.01 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss= 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [18]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model_history = model.fit(x_train, y_train, batch_size=64, epochs=12)
    model.save_weights('./author_model.h5')


AttributeError: module 'tensorflow' has no attribute 'Session'

In [19]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./author_model.h5')
    score = model.evaluate(x_test, y_test, batch_size=64)
    test_pred = model.predict(test['text'], batch_size=32)
    pred = model.predict(x_test, batch_size=32)


AttributeError: module 'tensorflow' has no attribute 'Session'

In [20]:
score


NameError: name 'score' is not defined

In [21]:
preds = decode(le, pred)
y_true = decode(le, y_test)


ValueError: A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

In [22]:
c= confusion_matrix(y_true=y_true, y_pred=preds)
sns.heatmap(c, annot=True);


NameError: name 'y_true' is not defined

In [23]:
print(classification_report(y_true=y_true, y_pred=preds))


NameError: name 'y_true' is not defined

In [24]:
submission = pd.read_csv('../input/sample_submission.csv')
submission.columns


Index(['id', 'EAP', 'HPL', 'MWS'], dtype='object')

In [25]:
test.head()


,id,text
0,id27251,"There seemed to be a void, and nothing more, a..."
1,id09612,This event caused many of those who were about...
2,id11943,"I hastily gave my consent to this arrangement,..."
3,id19526,It were useless to record the debate that foll...
4,id12931,"But how are we to judge of airs, and pronounce..."


In [26]:
k = pd.DataFrame(test_pred, columns=['EAP', 'HPL', 'MWS'])
submission['EAP'] = k['EAP']
submission['HPL'] = k['HPL']
submission['MWS'] = k['MWS']
submission.head()


NameError: name 'test_pred' is not defined

In [27]:
submission.to_csv('my_submission1.csv', index=False)
